In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [21]:
df = pd.read_csv("/home/jose/Programming/aiml/Data/houston-AQI-weather/filled-in-data/Oyster Creek C1607.csv")

df['Date'] = pd.to_datetime(df['Date'])

del df['Unnamed: 0']

df.head()

,Date,AQI,air_temp,dew_point_temp,sea_level_pressure,visibility,wind_speed
0,2017-01-01,31,20.121739,17.578261,1011.065217,13311.826087,2.618182
1,2017-01-02,39,22.243478,17.991304,1010.547826,13714.173913,4.960000
2,2017-01-03,45,17.178261,10.843478,1014.776190,11859.869565,2.457143
3,2017-01-04,16,11.856522,5.769565,1018.547826,16093.000000,4.136364
4,2017-01-05,11,12.904348,9.900000,1013.213043,14833.652174,3.000000


In [22]:
### AQI is shifted one day into the future to predict

df['AQI'] = df.AQI.shift(1)

df.head()

,Date,AQI,air_temp,dew_point_temp,sea_level_pressure,visibility,wind_speed
0,2017-01-01,NaN,20.121739,17.578261,1011.065217,13311.826087,2.618182
1,2017-01-02,31.0,22.243478,17.991304,1010.547826,13714.173913,4.960000
2,2017-01-03,39.0,17.178261,10.843478,1014.776190,11859.869565,2.457143
3,2017-01-04,45.0,11.856522,5.769565,1018.547826,16093.000000,4.136364
4,2017-01-05,16.0,12.904348,9.900000,1013.213043,14833.652174,3.000000


In [23]:
# Delete first and last rows

df = df.drop(index=df.index[[0, -1]])

df.head()

,Date,AQI,air_temp,dew_point_temp,sea_level_pressure,visibility,wind_speed
1,2017-01-02,31.0,22.243478,17.991304,1010.547826,13714.173913,4.960000
2,2017-01-03,39.0,17.178261,10.843478,1014.776190,11859.869565,2.457143
3,2017-01-04,45.0,11.856522,5.769565,1018.547826,16093.000000,4.136364
4,2017-01-05,16.0,12.904348,9.900000,1013.213043,14833.652174,3.000000
5,2017-01-06,11.0,6.639130,3.582609,1017.347826,15463.347826,7.186957


In [24]:
df['Day'] = df.Date.dt.dayofyear
df['Year'] = df.Date.dt.year - 2017


In [25]:
df.tail()

,Date,AQI,air_temp,dew_point_temp,sea_level_pressure,visibility,wind_speed,Day,Year
1514,2021-02-23,57.0,17.0,6.0,1022.0,10000.0,3.4375,54,4
1515,2021-02-24,57.0,20.0,16.0,1018.0,10000.0,4.3750,55,4
1516,2021-02-25,39.0,20.0,17.0,1019.0,9000.0,5.0000,56,4
1517,2021-02-26,30.0,21.0,18.0,1014.0,10000.0,5.3125,57,4
1518,2021-02-27,27.0,22.0,19.0,1013.0,10000.0,5.9375,58,4
